In [72]:
# Parameters
INPUT_FILE = './data/champ6_raw.csv'
MAX_ORDER = 10
SCORE_LIST = [13,8,5,3,2,1,1,1,1,1]
# SCORE_LIST = [8,5,3,2,1,0,0,0,0,0]
# SCORE_LIST = [8,5,3,2,1,1,1,1,1,1]

assert len(SCORE_LIST) == MAX_ORDER

In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# from scipy.cluster import hierarchy
# from sklearn.cluster import AgglomerativeClustering
from scipy.optimize import linear_sum_assignment
import networkx as nx

In [63]:
# ประทับเวลา	ที่อยู่อีเมล	ชื่อ-นามสกุล ภาษาไทย	ชื่อเล่น	รหัสนิสิต	ชั้นปี	Pair ที่สนใจลำดับที่ 1	Pair ที่สนใจลำดับที่ 2	Pair ที่สนใจลำดับที่ 3	Pair ที่สนใจลำดับที่ 4	Pair ที่สนใจลำดับที่ 5	Pair ที่สนใจลำดับที่ 6	Pair ที่สนใจลำดับที่ 7	Pair ที่สนใจลำดับที่ 8	Pair ที่สนใจลำดับที่ 9	Pair ที่สนใจลำดับที่ 10
COLUMNS = ['timestamp', 'email', 'name_th', 'nickname', 'id', 'year']
RANK_COLS = [f'rank_{i}' for i in range(1, 11)]

In [64]:
def replace_nan(data):
    """Replace NaN Value
        If a column contains numerical value, replace it with 0.
        Otherwise, replace it with 'UNKNOWN'"""
    
    df = data.copy()
    for col in df.columns:
        if df[col].dtype in ['int64', 'float64']:
            df[col].fillna(0, inplace = True)
        else:
            df[col].fillna('UNKNOWN', inplace = True)
    return df

def tier_multiply(score, tier):
    """Multiply scores of each candidate by his/her respective tier
    """
    new = score.copy()
    new['tier'] = tier
    new = new.apply(lambda x: x * x['tier'])
    return new

In [65]:
pre_df = pd.read_csv(INPUT_FILE)
pre_df.columns = COLUMNS + RANK_COLS
# pre_df.set_index('id', inplace=True)
meta_df = pre_df.loc[:, COLUMNS].set_index('id')
rank_df = pre_df.loc[:, ['id'] + RANK_COLS] #.set_index('id')
# tier = pre_df.pop('tier')
# tier.index = pre_df.ID
# tier.head()
# pre_df.head()
rank_df.head(2)

,id,rank_1,rank_2,rank_3,rank_4,rank_5,rank_6,rank_7,rank_8,rank_9,rank_10
0,6531301721,"Pair 7: คุณกฤตภาส สิริภัสสร (พี่ก้อง), คุณอภิน...","Pair 9: คุณพิชิต พงษ์ประเสริฐ (พี่ไปรท์), คุณว...","Pair 11: ดร.ศรุต วานิชพันธ์ุ (พี่รัมย์), คุณเป...","Pair 8: คุณไพโรจน์ กวียานันท์ (พี่บูน), คุณปัญ...","Pair 12: คุณศิริวัฒน์ วงศ์จารุกร (พี่เล้ง), คุ...","Pair 13: คุณวรภัทร ชวนะนิกุล (พี่เบนซ์), คุณวี...","Pair 2: คุณอดิศัย ธรรมร่มดี(พี่แซม), ดร.ชวพล จ...","Pair 17: คุณรัชดา ด่านพงษ์เจริญ (พี่อ้อ), คุณร...","Pair 3: นายอครินทร์ ภูรีสิทธิ์ (พี่ต้น), คุณถึ...","Pair 14: คุณชลิยา จุลโมกข์ สุชาโต (พี่แนท), คุ..."
1,6338202321,"Pair 12: คุณศิริวัฒน์ วงศ์จารุกร (พี่เล้ง), คุ...","Pair 9: คุณพิชิต พงษ์ประเสริฐ (พี่ไปรท์), คุณว...","Pair 14: คุณชลิยา จุลโมกข์ สุชาโต (พี่แนท), คุ...","Pair 15: คุณปกรณ์ ธรรมภิมุขวัฒนา (พี่เหน่ง), ค...",Pair 1: คุณจีรวัฒน์ สิทธิ์สันต์กุล (พี่เจมส์ จ...,"Pair 2: คุณอดิศัย ธรรมร่มดี(พี่แซม), ดร.ชวพล จ...","Pair 3: นายอครินทร์ ภูรีสิทธิ์ (พี่ต้น), คุณถึ...","Pair 4: คุณปพนธ์ มังคละธนะกุล (พี่ปะ), คุณกวีว...","Pair 5: คุณสุรัตน์ ประลองศิลป์ (พี่ปู), คุณโทน...","Pair 6: คุณสุธี อัสววิมล (พี่โหน่ง), คุณกิตติธ..."


In [66]:
# top pairs of pairs
# rank_df.groupby(['rank_1', 'rank_2'])['id'].nunique()
pairs_list = [['rank_1', 'rank_2'], ['rank_1', 'rank_3']]
top_ranks = []
for pairs in pairs_list:
    tmp = rank_df.loc[:, ['id'] + pairs]
    tmp['v1'] = tmp[pairs].min(axis=1)
    tmp['v2'] = tmp[pairs].max(axis=1)
    # tmp['v1'] = tmp[pairs[0]]
    # tmp['v2'] = tmp[pairs[1]]
    top_ranks.append(tmp[['id', 'v1', 'v2']])

top_ranks_df = pd.concat(top_ranks, axis=0)
# top_ranks_df.head()
top_ranks_count = top_ranks_df.groupby(['v1', 'v2'])[['id']].count().reset_index() \
                .rename(columns={'id': 'n_att'}) \
                .sort_values(by='n_att', ascending=False)
top_ranks_count.head()

,v1,v2,n_att
67,"Pair 2: คุณอดิศัย ธรรมร่มดี(พี่แซม), ดร.ชวพล จ...","Pair 4: คุณปพนธ์ มังคละธนะกุล (พี่ปะ), คุณกวีว...",13
69,"Pair 2: คุณอดิศัย ธรรมร่มดี(พี่แซม), ดร.ชวพล จ...","Pair 6: คุณสุธี อัสววิมล (พี่โหน่ง), คุณกิตติธ...",7
24,"Pair 12: คุณศิริวัฒน์ วงศ์จารุกร (พี่เล้ง), คุ...","Pair 7: คุณกฤตภาส สิริภัสสร (พี่ก้อง), คุณอภิน...",7
61,Pair 1: คุณจีรวัฒน์ สิทธิ์สันต์กุล (พี่เจมส์ จ...,"Pair 2: คุณอดิศัย ธรรมร่มดี(พี่แซม), ดร.ชวพล จ...",6
6,"Pair 10: คุณวชิระชัย คูนำวัฒนา (พี่โจโจ้), คุณ...","Pair 6: คุณสุธี อัสววิมล (พี่โหน่ง), คุณกิตติธ...",6


In [67]:
# n = 11
# top_ranks_count[(top_ranks_count['v1'].str.contains(f'Pair {n}')) | (top_ranks_count['v2'].str.contains(f'Pair {n}'))]

# Data Preprocessing

In [68]:
df = replace_nan(rank_df)
df.isnull().sum().sum()

0

In [73]:
melt_df = pd.melt(df, id_vars='id', var_name='rank', value_name='pair')
melt_df['rank'] = melt_df['rank'].str.replace('rank_', '').astype(int)
melt_df['pair'] = melt_df['pair'].apply(lambda x: x.split(':')[0].lower().replace(' ', '_'))

score_transform = {k: v for k, v in zip(range(1, MAX_ORDER + 1), SCORE_LIST)}
melt_df['score'] = melt_df['rank'].replace(score_transform)

pair_rank_df = pd.pivot_table(melt_df, values='score', columns='pair', index='id', aggfunc='max') # .fillna(-99.0)

# reorder columns
reordered_cols = [f'pair_{i}' for i in range(1, 19)]
pair_rank_df = pair_rank_df[reordered_cols]
pair_rank_df.head()

pair,pair_1,pair_2,pair_3,pair_4,pair_5,pair_6,pair_7,pair_8,pair_9,pair_10,pair_11,pair_12,pair_13,pair_14,pair_15,pair_16,pair_17,pair_18
id,,,,,,,,,,,,,,,,,,
6330002721,3.0,2.0,5.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,13.0,1.0,1.0,NaN,8.0,NaN
6330013621,3.0,5.0,1.0,NaN,NaN,13.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,2.0,NaN,NaN,8.0,1.0
6330022221,3.0,1.0,NaN,1.0,8.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,13.0,2.0,NaN,1.0,5.0,NaN
6330034821,NaN,1.0,1.0,NaN,NaN,3.0,13.0,5.0,NaN,1.0,1.0,2.0,NaN,NaN,8.0,NaN,1.0,NaN
6330038321,NaN,NaN,1.0,1.0,5.0,8.0,NaN,2.0,NaN,NaN,NaN,1.0,3.0,1.0,1.0,13.0,NaN,NaN


In [111]:
# save pair rank data
pair_rank_df.to_csv('./res/champ_6_pair_rank.csv')

In [17]:
# melt_df.groupby(['id', 'pair'])[['rank']].count().sort_values(by='pair', ascending=False)

# Correlation Matrix

In [70]:
# # pair_score = df[[col for col in df if 'pair' in col]]
# corr = pair_rank_df.corr().fillna(0)
# corr.index.name = ''
# corr_table = corr.unstack().reset_index()
# corr_table.columns = ['p1', 'p2', 'corr']

# # top correlation
# corr_table[corr_table['p1'] > corr_table['p2']].sort_values(by='corr', ascending=False).head(10)

# Dendrogram

In [58]:
# den = hierarchy.dendrogram(hierarchy.linkage(corr, method = 'ward'),
#                            labels = corr.index,
#                            leaf_rotation=90)
# plt.title('Dendrogram of weighted applicant ranks')

<h1 style="color:turquoise">Group Rooms with Hungarian Algorithm</h1>
<p>Ref: <a href="https://python.plainenglish.io/hungarian-algorithm-introduction-python-implementation-93e7c0890e15">Medium</a></p>

In [79]:
ROOMS = {
    'a': [1, 3, 14],
    'b': [2, 4, 11],
    'c': [5, 12, 13],
    'd': [6, 9, 10],
    'e': [7, 16, 17],
    'f': [8, 15, 18]
}

pair_to_room = {}
for room, pairs in ROOMS.items():
    for n in pairs:
        pair_to_room[f'pair_{n}'] = f'room_{room}'
# pair_to_room

In [85]:
# use melt_df
room_df_tmp = melt_df.copy()
room_df_tmp['room'] = room_df_tmp['pair'].map(pair_to_room)
room_df = pd.pivot_table(room_df_tmp, index='id', columns='room', values='score', aggfunc='sum').fillna(0)

In [86]:
room_df.head()

room,room_a,room_b,room_c,room_d,room_e,room_f
id,,,,,,
6330002721,9.0,4.0,14.0,0.0,8.0,1.0
6330013621,6.0,6.0,0.0,14.0,9.0,1.0
6330022221,5.0,2.0,22.0,1.0,6.0,0.0
6330034821,1.0,2.0,2.0,4.0,14.0,13.0
6330038321,2.0,1.0,9.0,8.0,13.0,3.0


In [91]:
# room_df_tmp[room_df_tmp.id == 6330002721].sort_values(by=['room', 'pair'])

In [107]:
# Sample data (replace this with your actual data)
preferences = room_df.values #.T
n_customers, n_menus = preferences.shape

# Create the cost matrix (convert preferences to negative values)
cost_matrix = -preferences

# Add constraints to ensure each menu is assigned to 10 customers
# Create an identity matrix for the constraint
constraint_matrix = np.eye(n_customers)
# Duplicate the identity matrix for each menu
constraint_matrix = np.tile(constraint_matrix, (1, n_menus))
# Set a high cost for violating the constraint (e.g., positive infinity)
cost_matrix = np.vstack([cost_matrix, 1e6 * np.ones((n_menus, n_menus))])

# Solve the assignment problem
row_indices, col_indices = linear_sum_assignment(cost_matrix)

# Interpret the results
assignments = dict(zip(row_indices, col_indices))
print(assignments)


{14: 1, 19: 2, 26: 4, 29: 0, 55: 3, 81: 5}


In [109]:
preferences[:5]

array([[ 9.,  4., 14.,  0.,  8.,  1.],
       [ 6.,  6.,  0., 14.,  9.,  1.],
       [ 5.,  2., 22.,  1.,  6.,  0.],
       [ 1.,  2.,  2.,  4., 14., 13.],
       [ 2.,  1.,  9.,  8., 13.,  3.]])

In [110]:
room_df.head()

room,room_a,room_b,room_c,room_d,room_e,room_f
id,,,,,,
6330002721,9.0,4.0,14.0,0.0,8.0,1.0
6330013621,6.0,6.0,0.0,14.0,9.0,1.0
6330022221,5.0,2.0,22.0,1.0,6.0,0.0
6330034821,1.0,2.0,2.0,4.0,14.0,13.0
6330038321,2.0,1.0,9.0,8.0,13.0,3.0
